In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import os

In [2]:
img_files = os.listdir("pdf2png")

In [19]:
data = {}
img_shape = None
count = 0
for img_file in img_files:
    img = cv2.imread("pdf2png/"+img_file,0)[600:1700,200:3400]
    img_shape = img.shape
    x = ['a_req_min' , 'a_req_max' , 'b_min' , 'b_max']
    with open("dilated/harris/del_"+img_file[:-4]+".npz","rb") as f:
        points = {i: np.load(f)[i] for i in x}
    data[count] = {"img":img, "points":points}
    count+=1

In [14]:
view_3d = np.ones((5*len(data)+2,img_shape[0], img_shape[1]))*255

In [ ]:
def offset_things(offsets, img1, img2):
    h, w = img1.shape
    for i in range(h):
        for j in range(w):
            img2[i,j] = img1[(i+h-offsets[0])%h, (j +w - offsets[1])%w]
    

In [16]:
already_done = None
count= 0 
for img_file in img_files:
    if already_done is None:
        view_3d[0,:,:] = data[img_file[:-4]]["img"]
        already_done = 1
        offsets = data[img_file[:-4]]["points"]
    else:
        view_3d[count*5,:,:] = data[img_file[:-4]]["img"]

    count += 1
    

In [18]:
h,w = img.shape

In [20]:

fig = plt.figure(figsize=(10, 10))
 
ax = plt.axes(projection='3d')

x = np.arange(0, h, 1)
y = np.arange(0, w, 1)

count = 0 
for i in range(len(data)):
    if count==0:
        offset_setting = (data[count]["points"]['a_req_min'],  data[count]["points"]['b_min'])
        for x in range(h):
            for y in range(w):
                if data[count]["img"][x,y]<250:
                    ax.scatter(x,y, count*5, 'o')
    else:
        offset_diff = (data[count]["points"]['a_req_min'],  data[count]["points"]['b_min'])
        offset_diff = (offset_diff[0]- offset_setting[0], offset_diff[1]- offset_setting[1])
        for x in range(h):
            for y in range(w):
                if data[count]["img"][x,y]<250:
                    ax.scatter((x+h-offset_diff[0])%h,(y+w-offset_diff[1])%w, count*5, 'o')
    count+=1
    
plt.show()

KeyboardInterrupt: 